<a href="https://colab.research.google.com/github/redrebbaa/SmartMeterVisualisation/blob/main/Image_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q jupyter-dash==0.3.0rc1 dash-bootstrap-components transformers
!pip install -q jupyter kaleido
import sys
!{sys.executable} -m pip install orca


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



In [ ]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
from transformers import AutoModelWithLMHead, AutoTokenizer
import plotly.graph_objs as go


In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib import pylab
from scipy import stats
import seaborn as sns
import PIL
from PIL import Image
import os
import datetime
import time
import torch
import plotly.express as px
from datetime import date
import re

In [ ]:
app = JupyterDash(__name__)
root = '/content/drive/MyDrive/Smart_Meters/house_'
HeatmapFig = html.Div(dcc.Graph(id='HeatmapFig'))

HouseOptions = []
xAxisOptions=[]
xAxisLabels = ['Minute', 'Hour', 'Day', 'Week']
ApplianceOptions =[]

for i in range(5):
  HouseOptions.append({'label': str(i+1), 'value': str(i+1)})
HouseMenu = html.Div(dcc.Dropdown(id='HouseMenu', options=HouseOptions,
                        placeholder = "Choose the houses that you want to view"))

ApplianceMenu = html.Div(dcc.Dropdown(id='ApplianceMenu', options=ApplianceOptions,
                        multi = True,
                        placeholder = "Choose the appliances that you want to view"))

for i in xAxisLabels:
  xAxisOptions.append({'label': i, 'value': i})

DateMenu = html.Div(dcc.DatePickerRange(
        id='DateMenu'))

SamplingMenu = html.Div(dcc.Input(id="SamplingMenu", type="number",
                         placeholder="Input the sampling period in seconds", 
                         min=6, step =6))

xAxisMenu = html.Div(dcc.Dropdown(id='xAxisMenu', options=xAxisOptions,
                        placeholder = "Choose the length of each block of the x axis"))

app.layout = html.Div([HeatmapFig, HouseMenu, DateMenu, ApplianceMenu, SamplingMenu, xAxisMenu])
@app.callback(
    Output(component_id='HeatmapFig', component_property='figure'),
    Input(component_id='SamplingMenu', component_property='value'),
    Input(component_id='xAxisMenu', component_property='value'),
    Input(component_id='HouseMenu', component_property='value'),
    Input(component_id='ApplianceMenu', component_property='value'),
    Input(component_id='DateMenu', component_property='start_date'),
    Input(component_id='DateMenu', component_property='end_date'))
def update_output_div(Period, xAxis, HouseNum, Appliances, StartDate, EndDate):
  StartDate = StartDate.split('-')
  EndDate = EndDate.split('-')
  TimeRange = [datetime.datetime(int(StartDate[0]), int(StartDate[1]), int(StartDate[2])).timestamp(), 
               datetime.datetime(int(EndDate[0]), int(EndDate[1]), int(EndDate[2])).timestamp()]
  TotalTime = TimeRange[1] - TimeRange[0]
  if xAxis == 'Minute':
    DataSize = (int(TotalTime/60), int(60/Period))
  elif xAxis == 'Hour':
    DataSize = (int(TotalTime/3600), int(3600/Period))
  elif xAxis == 'Day':
    DataSize = (int(TotalTime/86400), int(86400/Period))
  elif xAxis == 'Week':
    DataSize = (int(TotalTime/604800), int(604800/Period))
  heatmap = NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, HouseNum, Appliances, root)
  fig = go.Figure(data=go.Heatmap(z = heatmap))
  FigureLabel = ('The ' + ', '.join(Appliances) + ' data from house number '
  + HouseNum + ' for ' + xAxis + ' starting on day ' + str(TimeRange[0]) + 
  ' and ending on day ' + str(TimeRange[1]) )
  #fig.write_image('/content/drive/MyDrive/Smart_Meters/Figures/' + FigureLabel + '.jpeg')
  return fig

@app.callback(
    Output(component_id='ApplianceMenu', component_property='options'),
    Input(component_id='HouseMenu', component_property='value'))
def UpdateApplianceLabels(HouseNum):
  ApplianceOptions = []
  root = '/content/drive/MyDrive/Smart_Meters/house_'
  ApplianceLabels = ((pd.read_table(root + HouseNum + '/labels.dat',sep="\s+",names=['Num','Label'])).to_numpy())[:, 1]
  for i in ApplianceLabels:
    ApplianceOptions.append({'label': i, 'value': i})
  return ApplianceOptions

@app.callback(
    Output(component_id='DateMenu', component_property='min_date_allowed'),
    Output(component_id='DateMenu', component_property='max_date_allowed'),
    Output(component_id='DateMenu', component_property='initial_visible_month'),
    Input(component_id='HouseMenu', component_property='value'))
def UpdateDateMenu(HouseNum):
  root = '/content/drive/MyDrive/Smart_Meters/house_'
  File = root + HouseNum + '/ aggregate.dat'
  Data = (pd.read_table(File,sep="\s+",names=['t','PR'])).to_numpy()
  MinDate = date.fromtimestamp(Data[0,0])
  MaxDate = date.fromtimestamp(Data[-1,0])
  initial_visible_month = MinDate
  return MinDate, MaxDate, initial_visible_month

if __name__ == "__main__":
  app.run_server(host="127.0.0.1",port=8000, debug = True, mode='external')

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
def NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, HouseNum, Appliances, root):
  TotalDataPoints = int(TotalTime/Period)
  Data = np.zeros((int(TotalTime/6), (len(Appliances)+1)), int)
  for i in range(len(Appliances)):
    File = root + HouseNum + '/ ' + Appliances[i] + '.dat'
    AppData = ProcessData(File, TimeRange)
    Data[0:((np.shape(AppData))[0]), [0, i+1]] = AppData[:, [0, 1]]
  heatmap = CreateHeatmap(TotalDataPoints, np.sum((Data[:, 1:]),1), Data[:, 0], TimeRange[0], Period, DataSize)
  return heatmap

In [ ]:
def ProcessData(file, TimeRange):
  data = (pd.read_table(file,sep="\s+",names=['t','PR'])).to_numpy()
  data = data[(data[:,0] >= (TimeRange[0])) & (data[:,0] < (TimeRange[1]))]
  return data

In [ ]:
def CreateHeatmap(TotalDataPoints, DataValues, DataLabels, window_start, Period, DataSize):
    heatmap = np.zeros(TotalDataPoints)
    for i in range(TotalDataPoints):
      heatmap[i] = np.mean(DataValues[(DataLabels >= window_start + i*Period) & (DataLabels < (window_start) +((i+6)*Period))])
    heatmap = np.transpose(heatmap.reshape(DataSize))
    heatmap = np.repeat(heatmap, Period, 0)
    return heatmap

In [ ]:
start = [1352500095, 1352503695]
Period = 60
TotalTime = start[1] - start[0]
DataSize = (int(TotalTime/60), int(60/Period))
DataLength = 'hour'
HouseNum = '1'
Appliances = ['aggregate']
root = '/content/drive/MyDrive/Smart_Meters/house_'
heatmap = NILMHeatmap(start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, root)
fig = go.Figure(data=go.Heatmap(z = heatmap))
fig.show()
fig.write_image('/content/drive/MyDrive/Smart_Meters/Figures/WORKPLEASE.jpeg')

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



ValueError: ignored

In [ ]:
def PlotHeatmap(heatmap):
  fig = go.Figure(data=go.Heatmap(z = heatmap))
  fig.show()

  """
  Matplotlib plotting stuff

  plt.figure(FigureNum)
  plt.title(FigureLabel)
  plt.xlabel('')
  plt.ylabel('Seconds')
  plt.axis('on')
  sns.heatmap(heatmap,cmap='icefire', cbar = True)
  plt.savefig('/content/drive/MyDrive/Smart_Meters/Figures/' + FigureLabel)
  plt.show(FigureNum)
  plt.close(FigureNum)
  """

In [ ]:
main()

In [ ]:
def main():
  root = '/content/drive/MyDrive/Smart_Meters/house_'
  start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, MultiHouse = UserInput(root)
  if MultiHouse:
    for i in HouseNum:
      NILMHeatmap(start, Period, TotalTime, DataSize, DataLength, i, Appliances, root)
  else:
    NILMHeatmap(start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, root)

In [ ]:
def UserInput(root):
  HouseNum = None
  Period = 1
  DataLength = 'z'
  TotalTime = 0
  start = [0, 0]
  Appliances = [None]
  NumAppliances = 1

  while HouseNum not in ['1', '2', '3', '4', '5']:
    HouseNum = str(input('Input the house number between 1 and 5: '))
  MultiAppliance = input('Would you like to input data from multiple appliances? Yes/No ')
  if MultiAppliance in 'Yes':
    MultiAppliance = True
    NumAppliances = int(input('How many appliances would you like to input? '))
    Appliances = [None] * NumAppliances
  
  ApplianceLabels = ((pd.read_table(root + HouseNum + '/labels.dat',sep="\s+",names=['Num','Label'])).to_numpy())[:, 1]
  for i in range(NumAppliances):
    while Appliances[i] not in ApplianceLabels:
      Appliances[i] = str(input('Choose the appliance name from the list: ' + ', '.join(ApplianceLabels) + ': '))
  
  MultiHouse = input('Would you like to compare across multiple houses? Yes/No ')
  if MultiHouse in 'Yes':
      MultiHouse = True
      HouseNum = [HouseNum]
      for i in range(5):
        ApplianceLabels = ((pd.read_table(root + str(i+1) + '/labels.dat',sep="\s+",names=['Num','Label'])).to_numpy())[:, 1]
        if (set(Appliances).issubset(ApplianceLabels)) and (str(i+1) not in HouseNum):
          HouseNum.append(str(i+1))
  else:
    MultiHouse = False
  
  while Period % 6 != 0:
    Period = int(input('Input the sampling period: '))
  while TotalTime == 0:
    if DataLength in 'Hour':
      TotalTime = 3600
      DataSize = (60, int(60/Period))
      DataLength = 'hour'
    elif DataLength in 'Day':
      TotalTime = 86400
      DataSize = (24, int(3600/Period))
      DataLength = 'day'
    elif DataLength in 'Week':
      TotalTime = 604800
      DataSize = (7, int(86400/Period))
      DataLength = 'week'
    elif DataLength in 'Month':
      TotalTime = 2678400
      DataSize = (31, int(86400/Period))
      DataLength = 'month'
    elif DataLength in 'Year':
      TotalTime = 31536000
      DataSize = (365, int(86400/Period))
      DataLength = 'year'
    else:
      DataLength = input('Choose the length of time: Hour, Day, Week, Month, Year  :   ')
  start[0] = (int(input('Input the start day: ')))
  if DataLength == 'hour':
    start[1] = (int(input('Input the start hour: ')))
    DataLength = 'an ' + DataLength
  else:
    DataLength = 'a ' + DataLength
  return start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, MultiHouse